In [ ]:
from  lib.project_functions import *

In [ ]:
data = loading_random_data()
for image in data:
    image_show(image_preprocessing(image))

In [ ]:
def remove_duplicate_images_by_name(path_to_Train = "data\Train"):

    for folder in tqdm(os.listdir(path_to_Train)):
        folder_path = os.path.join(path_to_Train, folder)

        for image in os.listdir(folder_path):
            parts = image.split("_")

            if len(parts) > 3:
                image_path = os.path.join(folder_path, image)
                os.remove(image_path)

remove_duplicate_images_by_name(path_to_Train )

# imega transformation

In [ ]:
def edit_training_data( path_to_Train="data\Train", path_to_edited_data ="data\edited_training_data"):

    name_folders = os.listdir(path_to_Train)
    os.makedirs(path_to_edited_data, exist_ok=True)
    
    for name_folder in tqdm(name_folders):
        path_to_train_data = os.path.join(path_to_Train, name_folder)
        path_to_new_data = os.path.join(path_to_edited_data, name_folder)
    
        os.makedirs(path_to_new_data, exist_ok=True)
        
        path_images = os.listdir(path_to_train_data)
    
        for image in path_images:
            train_path = os.path.join(path_to_train_data, image)
            new_data_path = os.path.join(path_to_new_data, image)
            
            if not os.path.exists(new_data_path):
                img = cv.imread(train_path)
                
                if img is not None:
                    preprocessed_img = image_preprocessing(img)
                    cv.imwrite(new_data_path, preprocessed_img) 
                
    
edit_training_data()       

# Train data

In [ ]:
generator_batch_size = 32
generator_image_size = (128, 128)

# Создайте экземпляр ImageDataGenerator
image_generator = ImageDataGenerator(
    rescale=1.0/255.0,  # Масштабирование значений пикселей в диапазон [0, 1]
    preprocessing_function= image_preprocessing, # Пользовательская функция предобработки
    rotation_range=40,      # Случайное вращение изображения на угол до 40 градусов
    width_shift_range=0.2,  # Случайное смещение изображения по горизонтали на 20% от ширины
    height_shift_range=0.2, # Случайное смещение изображения по вертикали на 20% от высоты
    shear_range=0.2,        # Случайное сдвигание изображения (изогнутость)
    zoom_range=0.2,         # Случайное увеличение масштаба изображения
    horizontal_flip=True,   # Случайное горизонтальное отражение
    fill_mode='nearest'     # Заполнение пикселей после трансформаций
)

train_data_generator = image_generator.flow_from_directory(
    'data/Train',
    target_size=generator_image_size,
    batch_size=generator_batch_size,
    class_mode='categorical'  # Если у вас несколько классов
)

In [ ]:
data_from_batch = next(train_data_generator)

In [ ]:
# image_show(data_from_batch[0][31]) 

# Validation data

In [ ]:
validation_data_generators()

In [ ]:
validation_image_generator = ImageDataGenerator(
    rescale=1.0/255.0,  
    preprocessing_function= image_preprocessing, 
    rotation_range=40,      
    width_shift_range=0.2,  
    height_shift_range=0.2, 
    shear_range=0.2,        
    zoom_range=0.2,         
    horizontal_flip=True,   
    fill_mode='nearest'     
)

validation_data_generator = validation_image_generator.flow_from_directory(
    'data/Validation',
    target_size=generator_image_size,
    batch_size=generator_batch_size,
    class_mode='categorical'  
)

# Test data

In [ ]:
# Укажите путь к папке, из которой вы хотите удалить фотографии
folder_path = 'data/Test'

# Получите список всех файлов в папке
files = os.listdir(folder_path)

# Пройдитесь по списку файлов и удалите файлы с именем "00000_1"
for file_name in files:
    if "_" in  file_name :  # Замените '00000_1.jpg' на желаемое имя файла
        file_path = os.path.join(folder_path, file_name)
        os.remove(file_path)
        print(f'File {file_name} has been deleted.')

print('Removal is complete.')

In [ ]:
def test_data_generator():
    test_data_dir = 'data/Test'
    test_data_classes = pd.read_csv("data/Test.csv", usecols= ["ClassId","Path"])
    test_image_files = os.listdir(test_data_dir)
   
    for image_file in test_image_files:
        image_path = os.path.join(test_data_dir, image_file)
        image = cv.imread(image_path)  # Загрузка изображения с использованием OpenCV
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Преобразование цветового пространства
        image = cv.resize(image, generator_image_size)  # Изменение размера изображения
        image = image_preprocessing(image)  # Применение пользовательской предобработки
        
        class_label = int(test_data_classes.loc[image_file == test_data_classes["Path"].str.split("/").str.get(1), "ClassId"])
        
        yield image, (class_label)

test_ds = tf.data.Dataset.from_generator(
     test_data_generator,
     output_signature=(
         tf.TensorSpec(shape=(*generator_image_size,3), dtype=tf.float32),
         tf.TensorSpec(shape=(), dtype=tf.int32)))

In [ ]:
test_ds = test_ds.batch(generator_batch_size)

In [ ]:
for images, labels in test_ds.take(1):  # Взять первый батч из датасета
    print("Images shape:", images.shape) # Форма батча изображений
    plt.imshow(images[0])
    plt.show()
    print("Labels shape:", labels.shape)
    print(labels[0])

data generator 

In [ ]:
def generator(train:bool, test:bool):

    generator_batch_size = 32
    generator_image_size = (128, 128)
    
    # Create an instance of ImageDataGenerator for training data
    image_generator = ImageDataGenerator(
        rescale=1.0/255.0,  # Scale pixel values to the range [0, 1]
        preprocessing_function= image_preprocessing if image_preprocessing else None,  # Custom preprocessing function
        rotation_range=40,      # Random rotation up to 40 degrees
        width_shift_range=0.2,  # Random horizontal shift of 20% of the width
        height_shift_range=0.2, # Random vertical shift of 20% of the height
        shear_range=0.2,        # Random shear transformation
        zoom_range=0.2,         # Random zoom
        horizontal_flip=True,   # Random horizontal flip
        fill_mode='nearest'     # Fill mode after transformations
    )
    
    # Create a data generator for training data
    train_data_generator = image_generator.flow_from_directory(
        'data/Train',
        target_size=generator_image_size,
        batch_size=generator_batch_size,
        class_mode='categorical'  # For multi-class classification
    )

    validation_image_generator = ImageDataGenerator(
        rescale=1.0/255.0,  
        preprocessing_function= image_preprocessing if image_preprocessing else None, # image_preprocessing, 
        rotation_range=40,      
        width_shift_range=0.2,  
        height_shift_range=0.2, 
        shear_range=0.2,        
        zoom_range=0.2,         
        horizontal_flip=True,   
        fill_mode='nearest'     
    )
    
    validation_data_generator = validation_image_generator.flow_from_directory(
        'data/Validation',
        target_size=generator_image_size,
        batch_size=generator_batch_size,
        class_mode='categorical'  
    )

    def test_data_generator():
        test_data_dir = 'data/Test'
        test_data_classes = pd.read_csv("data/Test.csv", usecols=["ClassId", "Path"])
        test_image_files = os.listdir(test_data_dir)

        for image_file in test_image_files:
            image_path = os.path.join(test_data_dir, image_file)
            image = cv.imread(image_path)  # Load the image using OpenCV
            image = cv.resize(image, generator_image_size)  # Resize the image
            image = image_preprocessing(image)  # Apply custom preprocessing

            class_label = int(test_data_classes.loc[image_file == test_data_classes["Path"].str.split("/").str.get(1), "ClassId"])

            yield image, class_label
            
    # Create a tf.data.Dataset for testing data
    test_dg = tf.data.Dataset.from_generator(
        test_data_generator,
        output_signature=(
            tf.TensorSpec(shape=(*generator_image_size, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )
    )
    test_dg = test_dg.batch(generator_batch_size)

    if train and test:
        return train_data_generator, validation_data_generator,  test_dg
    elif train:
        return train_data_generator, validation_image_generator
    elif test:
        return test_dg


    
x, v, y =  generator(train=True, test= True)

In [ ]:
data_from_batch = next(x)
image_show(data_from_batch[0][31]) 

In [ ]:
for images, labels in y.take(1):  # Взять первый батч из датасета
    print("Images shape:", images.shape) # Форма батча изображений
    plt.imshow(images[0])
    plt.show()
    print("Labels shape:", labels.shape)
    print(labels[0])